## 2.1 Processes ##
1. Process: an abstraction of a running program.

2. Pseudoparallelism v.s. multiprocessor 

3. In each CPU, there is always one process running.

### 2.1.1 The Process Model ###
1. In this model, all the runnable software on the computer, sometimes including the operating system, is organized into a number of ***sequential processes(processes)***.

2. A process is an instance of an executing program, including the current values of the program counter, registers, and variables. Conceptually, each process has its own virtual CPU. 

3. Processes must not be programmed with built-in assumptions about timing.

4. Process v.s. program
        - a ***process*** is an activity of some kind. It has a program, input, output, and a state.
        - a single processor may be shared among several processes, with some scheduling algorithm being accustomed to determine when to stop work on one proces and service a different one.
        - in contrast, a ***program*** is something that may be stored on disk, not doing anything.

### 2.1.2 Process Creation ###
1. Four principle events cause processes to e created:
        - System initialization.
        - Execution of a process-creation system call by a running process.
        - A user request to create a new process.
        - Initiaiton of a batch job.
        
2. Deamons: processes that stey in the background to handle some activity.

3. Technically, in all cases, a new process is created by having an existing process execute a process creation system call.

4. `fork` in the only one system call to create a new process in UNIX. This call creates an exact clone fo the calling process. After that, the child process executes `execve` or a similar system call to change its memory image and run a new program.

5. In Windows, in contrast, s single Win32 function call, `CreateProcess`, handles both process creation and loading the correct program into the new process.

6. In both UNIX and Windows systems, after a process is created, the parent and child have their own distinct address spaces. NO WRITABLE MEMORY IS SHARED.


### 2.1.3 Process Termination ###
1. Conditions where new process will terminate:
        - Normal exit (voluntary). E.g. the process has done its work
        - Error exit (voluntary). E.g. the process discovers a fatal error
        - Fatal error (involuntary). E.g. caused by the process, bugs
        - Killed by another process (involuntary). E.g. (by system call `kill`)
        
### 2.1.4 Process Hierarchies ###
1. In some systems, when a process creates another process, the parent process and child process continue to be associated in certain ways. The child process can itself create more processes, forming a process hierarchy.

2. A process has only one parent but can have multiple chilren.

3. In UNIX, a process and all of its children and further descendants together form a process group.

4. Unlike in Windows, process in UNIX cannot disinherit their children.

### 2.1.5 Process States ###
1. Three states a process may be in:
        - Running (actually using the CPU at that instant).
        - Ready (runnable; temporarily stopped to let another process run).
        - Blocked (unable to run until some external event happends).

2. Four possible transitions among these three states:
        - Process blocks for input.
        - Scheduler picks another paroacess.
        - Scheduler picks this process.
        - Input becomes available.
    
3. The lowest layer of a process-structured operating system handles interrupts and scheuling. Above taht layer are sequential processes.

### 2.1.6 Implementation of Processes ###
1. The operating system maintains a table (an array of structures), called the **process table/process control blocks**. This entry contains import information abotu the process' state, i.e., its program counter, stack pointer, memory allocation, the status of its open files, its accouting and scheduling information.

2. Associated with each I/O class is a location (typically at a fixed location near the bottom of memory) called the **interrupt vector**. It contains the address of the interrupt service procedure. 

### 2.1.7 Modeling Multiprogramming ###
1. Skeleton of what the lowest level of the operating system does when an interrupt occurs:
        a. Hadware stacks program counter, etc.
        b. Hardware loads new program counter from interrupt vector.
        c. Assembly-language procedure saves registers.
        d. Assembly-language procedure sets up new stack.
        e. C interrupt service runs (typically reads and buffers input)
        f. Scheduler decides which process is to run next.
        g. C procedure returns to the assembly code.
        h. Assembly-language procedure starts up new current process.
        
2. Degree of multiprogramming : don't quite get this 

The result of using pop-up threads is that the latency between message arrival and the start of processing can be made very short.

## 2.2 Threads ##
In traditional operating systems, each process has an address space and a single ghread of control. In fact, thatis almost the definition of a process. Nevertheless, in many situations, it is desirable to have multiple threads of control in the same address space running in quasi-parallel, as though they were (almost) separate processes (except for the shared address space).

### 2.2.1 Thread Usage ###
1. Threads: miniprocesses running in a process

2. Reasons to have threads:
        - In many aplications, multiple activities are going on at once. Some of these may block from time to time.
        - Threads have the ability for the parallel entities to share an address space and all of its data among themselves.
        - Threads are lighter weight than processes, thus they are easier (i.e., faster) to create and destroy.
        - Threads yield no performance gain when all of them are CPU bound, but when there is substantial computing and also substantial I/O, having threads allows these activities to overlap, thus speeding up the application.
        - Threads are useful on systems with multiple CPUs, where real parallelism is possible.
        
3. Example: Word processors: one thread listening to user input, one thread reformatting files, and one backup the whole file periodically.

4. Finite-state machine: each computation has a saved state, and there exists some set or events that can occur to change the state.

5. Three ways to construct a server:
        **Model**                  **Characteristics**
        Threads                     Parallelism, blocking system calls
        Single-threaded process     No parallelism, blocking system calls
        Finite-state machine        Parallelism, nonblocking system calls, interrupts

6. Example: an application that processes very large amounts of data. One input thread, one processing thread, one output thread.

### 2.2.2 The Classical Tread Model ###
1. The process model is based on two independent concepts: resource grouping and execution.

2. One way of looking at a process is that it is a way to group related resources together. A process has an address space containing program text and data, as well as other resources. These resources may include open files, child processes, pending alarms, signal handlers, accouting information, and more. By putting them together in the form of a process, they can be managed more easily.

3. The other concept a process has is a thread of execution. Process are used to group resources together, threads are the entities scheduled for execution on the CPU.

4. What treads add to the process model is to allow multiple executions to take place in the same process enviroment, to a large degree independent of one another.

5. When a multithreaded process is run on a single-CPU system, the threads take turns running.

6. Items shared by all threads in a process and items private to each thread:
        **Per-process items**            **Per-thread items**
        Address space                    Program counter
        Global variables                 Registers
        Open files                       Stack
        Child processes                  State
        Pending alarms
        Signals and signal handlers
        Accouting information
        
7. States of a thread: running; blocked; ready; terminated

8. `thread_create`. When multithreading is present, processes usually start with a single thread present. This ghread ha sthe ability to create new threads by calling this library procedure.

9. `thread_exit`. When a tread finishes its work.

10. `thread_join`. One thread can wait for a (specific) thread to exit by calling this procedure.

11. `thread_yield`. Allows a thread to voluntarily give up the CPU to let another thread run.

### 2.2.3 POSIX Threads ###
1. Pthreads: To make it possible to write portable threaded programs, IEEE has defined a standard to threads in IEEE standard 1003.1c. The threads package it defines is called **Pthreads**.

### 2.2.4 Implementing Threads in User Space ###
1. Advantages: 
        - A user-level threads package can be implemented on an operating system that does not support threads.
        - They allow each process to have its own customized scheduling algorithm.

2. When threads are managed in user space, each process needs it own private **thread table** to keep track of the threads in that process.

3. If the program calls or jumps to an instruction that is not in memory, a **page fault** occurs and the operating system will go and get the missing instruction (and its neighbors) from disk.

4. Disavantages:
        - The problem of hwo blocking system calls are implemented.
        - If a thread starts runing, no other thread in that process will ever run unless the first thread voluntarily gives up the CPU.
        - Programmers generally want threads precisely in application where the threads block often, as, for example, in a multithreaded Web server.


### 2.2.5 Implementing Threads in the Kernel ###
1. The kernel's thread table holds each thread's registers, state, and other information.

2. All calls that might block a thead are implemented as system calls, at consideraly greater cost than a call to a run-time system procedure.

3. When a thread blocks, the kernel, can run either another thread from the same process (if one is ready) or a thread from a different process.

4. Problems kernel threads can't solve perfectly:
        - What happens when a multithreaded process forks?
        - When a signal comes in (signals are sent to processes, not to threads), which thread should handle it?
        
### 2.2.6 Hybrid Implementation ###
Use kernel-level threads and then multiplex user-level threads onto some or all of them. 

### 2.2.7 Scheduler Activations ###
1. Approach divised by Anderson et al. (1988). The goals of the scheduler activation work are to mimic the functionality of kernal threads, but with the better performance and greater flexibility usually associated with threads packages implemented in user space.

2. User threads should not have to make special nonblocking system calls or check in advance if it is safe to make certain system calls. Nevertheless, when a thread blocks on a system call or on a page fault, it should be possible to run other threads within the same process.

3. When scheduler activations are used, the kernel assigns a certain number of virtual processors to each process and lets the (user-space) run-time system allocate threads to processors. 

4. upcall

### 2.2.8 Pop-Up Threads ###
1. In the example of requests for service. The arrival of a message causes the system to create a new thread to handle the message. Such a thread is called **pop-up thread**.

2. The result of using pop-up threads is that the latency between message arrival and the start of processing can be made very short.

### 2.2.9 Making Single-Threaded Code Multithreaded ###
1. Problems need consideration:
        - private global variables
        - many library procedures are not reentrant (meaning they are not designed to have a second call made to any given procedure while a previous call has not yet finished
        - some signals are logically thread specific, whereas others are not
        - stack management (when a process' stack overflows, the kernel just provides that process with more stack automatically. When a process has multiple threads, it must also have multiple stacks. If the kernel is not aware of all these stacks, it cannot grow automatically upon stack falt.)

## 2.3 Interprocess Communication ##
1. InterProcess Communication, also called IPC.

2. Three main issues:
        - how one process can pass information to another
        - how to make sure two or more processes do not get in each other's way
        - how to deal with proper sequencing when dependencies are present

3. The above issues and solutions also apply to threads.

### 2.3.1 Race Conditions ###
1. ***Race conditions***: situations where two or more processes are reading or writing some shared data and the final result depends on who runs precisely when.

2. Example: print spooler, where process A and B wants to write in the next free slot. Just the minute process A reads and stores the value, a clock interrupt occurs and the CPU decides A has run long enough and witches to process B. Process B read and write into the slot and erase what process A has written.

### 2.3.2 Critical Regions ###
1. ***Mutual exclusion***: some way of making sure that if one process is using a shared variable or file, the other process will be excluded from doing the same thing.

2. ***Critical region/critical section***: the part of the program where the shared memory is accessed.

3. To aviod condition races, we have to make sure no two processes are in their critical regions at the same time.

4. A good solution for race conditions meets the following 4 requirements:
        - No two processes may be simultaneously inside their critical regions.
        - No assumptions may be made about speeds or the number of CPUs.
        - No process running outside its critical region may block any process.
        - No process should have to wait forever to enter its critical region.
        
### 2.3.3 Mutual Exclusion with Busy Waiting ###
This section examines various proposals for achieving mutual exclusion.

#### Disabling Interrupts ####
1. Definition: to have each process disable all interrupts just after entering its critical region and re-enable them just before leaving it.

2. Disabling intterupts is often a useful technique within the operating system itself but is not appropriate as a general mutual exlusion mechanism for user processes. 

3. However, this method won't work because 
        - situations like one process who disables interrupts but never turn them on again will crash the whole system
        - for multiprocessor, disabling interrupts ONLY affects the CPU that executed the `disable` instruction
        
#### Lock Variables ####
1. Definition: having a single, shared (lock) variable, initially 0 and is set to 1 when it is being read by a process.

2. This won't work at all.

#### Strict Alternation ####
1. ***busy waiting***: Continuously testing a variable until some value appears. It should usually be avioded, since it watstes CPU time. Only tehre is a reasonable expectation taht the wait will be short is busy waiting used.

2. ***spin lock***: A lock that uses busy waiting.

3. Definition: there is integer variable `turn` to keeps track of whose turn it is to enter the critical region. 

```C
/* Process 0 */
while (TRUE) {
    while (turn != 0)  /* loop */
    critical_region();
    turn = 1;
    noncritical_region();
}
```

```C
/* Process 1 */
while (TRUE) {
    while (turn != 1)  /* loop */
    critical_region();
    turn = 0;
    noncritical_region();
}
```

4. However, process running outside its critical region may block other process.

#### Peterson's Solution ####
<img src="https://img.vim-cn.com/d2/53ae49c6d0debdeede0eccc6fe4879c8e6edc8.png">


#### The TSL Instruction ####
1. `TSL RX.LOCK`(Test and Set Lock): it reads the contents of the memory wordk *lock* into register RX and tehm stores a nonzero value at the memory address *lock*. The operations of reading the word and storing into it are guaranteed to be indivisible - no other processor can access the memory word until the instruction is finished. The CPU executing the TSL instruction *locks the memory bus* to prohibit other CPUs from accessing memory until it is done.


3. Entering and leaving a critical region using the TSL instruction:
        enter_region:
            TSL REGISTER,LOCK      |copy lock to register and set 
            CMP REGISTER,#0        |was lock zero? 
            JNE enter_region       |if it was not zero, lock was set, so loop
            RET                    |return to caller; critical region entered
        leave_region:
            MOVE LOCK,#0           |store a 0 in lock
            RET                    |return to caller
            
4. XCHG: an alternative instruction to TSL which exchanges the contents of two locations atomically and is used by all Intel x86 CPUs.


### 2.3.4 Sleep and Wakeup ###
1. ***priority inversion problem***: A scenario in scheduling in which a high priority task is indirectly preempted by a low priority task effectively inverting the relative priorities of the two tasks.

2. The producer-consumer problem
        - wakeup waiting bit

### 2.3.5 Semaphores ###
1. Definition: proposed by E. W. Dijkstra (1965), is an iteger variable to count the number of wakeups saved for future use. 

2. A semaphore has two operations: `down` and `up`. `down` checks if the value is greater than 0 and if so, it decrements the value and then continues. It the value is already 0, the process is put to sleep. `up` increment the value to 1.

3. It is guaranteed that once a sempahore operation has started, no other process can access the semaphore until the operation has completed or blocked. It's called ***atomic action***. The operation of incrementing the semaphore and waking up one process is also atomic.

4. We can use semaphores to solve the producer-consumer problem.

```C
#define N 100         /* number of slots in th buffer */
typedef int semaphore;/* a special kind of int */
semaphore mutex = 1;  /* control access to critical region */
semaphore empty = N;  /* counts empty buffer slots */
semaphore full = 0;   /* counts full buffer slots */

void producer(void)
{
    int item;
    while (TRUE) {
        item = produce_item();
        down(&empty);
        down(&mutex);
        insert_item(item);
        up(&mutex);
        up(&full);
    }
}

void consumer(void)
{
    int item;
    while (TRUE) {
        down(&full);
        down(&mutex);
        remove_item(item);
        up(&mutex);
        up(&empty);
        consume_item(item);
    }
}
```

### 2.3.6 Mutexes ###
1. A **mutex** is a shared variable that can be in one of two states: unlocked and locked. They are easy and efficient to implement, which makes them especially useful in thread packages that are implemented entirely in user space.

2. Implementation of *mutex_lock* and *mutex_unlock*

```C
mutext_lock:
    TSL REGISTER,MUTEX
    CMP REGISTER,#0
    JZE ok
    CALL thread_yield
    JMP mutex_lock
ok: RET

mutex_unlock:
    MOVE MUTEX,#0
    RET
```

3. Difference between `enter_region` and `mutex_lock`:
        - There is no busy waiting in `mutex_lock` because there is no clock that stops threads that have run too long; while in process, there is a clock and scheduler for each run of the process.
        - When a thread fails to acquire a lock, it calls `thread_yield` to give up the CPU to another thread.
        - Neither `mutex_lock` or `mutex_unlock` requires ay kernel call.

#### Futexes ####
1. Futex, "fast user space mutex", is a feature of Linux that implements basic locking (much like a mutex) but avoids dropping into the kernel unless it really has to, since switching to the kernel and back is quite expensive.

#### Mutexes in Pthreads ####
1. Some of the Pthreads calls relating to mutexes
        `Pthread_mutex_init`    |  Create a mutex
        `Pthread_mutex_destory` |  Destory an exiting mutex
        `Pthread_mutex_lock`    |  Acquire a lock or block
        `Pthread_mutex_trylock` |  Acquire a lock or fail
        `Pthread_mutex_unlock`  |  Release a lock
        
2. ***Condition variables*** allow threads to lock due to some conditions not being met. They also allow the threads waiting and blocking to be done atomically.

3. Condition variables and mutexes are always used together. For one thread to lock a mutex, then wait on a conditional variable when it cannot get what it needs. Eventually another thread will signal it and ti can continue.

4. Condition variables (unlike semaphores) have no memory.

5. Some of the Pthreads calls relating to condition variables
        `Pthread_cond_init`
        `Pthread_cond_destory`
        `Pthread_cond_wait`      |  Block waiting for a signal
        `Pthread_cond_signal`    |  Signal another thread and wake it up
        `Pthread_cond_broadcast` |  Signal multiple threads and wake all of them
        
### 2.3.7 Monitors ###
1. Brinch Hansen(1973) and Hoare (1974) proposed a higher-level synchronization primitive called a ***monitor***. 

2. A monitor is a collection of procedures, variables, and data structures taht are all grouped together in a special kind of mudle or package. It's a language concept.

3. Processes may call the procedures in a monitor whenever they want to, buthey cannot directly access the monitor's internal data structures from procedures declared outside the monitor.

4. Only one process can be active in a monitor at any isntant. By turning all the critical regions into monitor procedures, no two processes will ever execute their critical regions at the same time.

5. By adding the keyword `synchronized` to a method declaration, Java guarantees that once any thread has started executing that method, no other thread will be allowed to start executing any other `synchronized` method.

6. Semaphores are too low level and monitors are not usable except in a few programming languages. Also, none of the primitives allow information exchange between machines.

### 2.3.8 Message Passing###
1. Uses two primitives, `send` and `receive` system calls.

2. Message passing is commonly used in parallel programming systes. 

3. MPI (Message-Passing Interface)

### 2.3.9 Barriers ###
1. Barriers are intended for groups of processes, in scenarios where aplications are divided into phrases and have a rule that no process may proceed into the next phrase until all processes are ready to proceed to the next phrase. A barrier is often placed at the end of each phrase.

### 2.3.10 Avoiding Locks: Read-Copy-Update ###



## 2.4 Scheduling ##
1. Many of the same issues that apply to process scheduling also apply to thread scheduling but some are different.

2. When the kernel manages threads, scheduling is usually done per thread, with little or or regard to which process the thread belongs.

### 2.4.1 Introduction to Scheduling ###
1. Most programs for personal computers are limited by the rate at which the user can present input (by typing or clicking), not by the rate the CPU can process it.

2. In addition to picking the right process to run, the scheduler also has to worry about making efficient use of the CPU because process switching is expensive.

#### Process Behavior ####
1. I/O is when a process enters the blocked state waiting for an external device to complete its work.

2. Compute-bound processes typically have long CPU bursts and thus infrequent I/O waits, whereas I/O-bound processes have short CPU bursts and thus frequent I/O bound. **The key factor is the length of the CPU bursts**.

#### When to Schedule ####
1. Nonpreemptive scheudling algorithm: picks a process to run and thenjsut lets it run until it blocks (either on I/O or waiting for another process) or voluntarily releases the CPU.

2. Preemptive scheduling algorithm: picks a process and lets it run for a maximum of some fixed time.

#### Categories of Scheduling Algorighm ####
1. Batch systems usually use nonpreemptive algorithms.

2. Environments with interative users, especially servers, often use preemptive algorithms.

3. Real time environments often use nonpreemptive algorithms.

#### Scheduling Algorithm Goals ####
        All systems
            - Fairness - giving each process a fair share of the CPU
            - Policy enforcement - seeing that stated policy is carried out
            - Balance - keeping all parts of the system busy
        
        Batch systems
            - Throughput - maximize jobs per hour
            - Turnaround time - minimize time between submission and termination
            - CPU utilization - keep the CPU busy all the time
            
        Interactive systems
            - Response time - respond to requests quickly
            - Proportionality - meet users' expections
        
        Real-time systems
            - Meeting deadlines - aviod losing data
            - Predictability - avoid quality degradation in multimedia systems
            
### 2.4.2 Scheduling in Batch Systems ###
#### First-Come, First-Served #### 
1. Advantage: easy to understand; easy to program

2. Disadvantage: can't efficient deal with large amout of I/O bound processes

3. Data structure: linked list

#### Shortest Job First ####
1. Provided there are four jobs: a, b, c, d and their runtime are 8, 4, 4, 4, respectively. Runing them in this order will produce a average turnaround time of 14 minutes. But we run these task in order b, c, d, a, the average turnaround time will be 11 minutes.

2. This algorithm is optimal ONLY when all the jobs are available simultaneously.

#### Shortest Remainign Time Next ####
1. The run time has to be known in advance.


### 2.4.3 Scheduling in Interactive Systems ###
#### Round-Robin Scheduling ####
1. The oldest, ismplest, fiarest, and most widely used algorithm.

2. Each process is assigned a time interval, called its ***quantum***, during which it is allowed to run.

3. Setting the quantum too short causes too many process switches and lowers the CPU efficiency, but setting it too long may cause poor reponse to short interactive requests. 

4. A quantum around 20-50 msec is often a reasonable compromise.

#### Priority Scheduling ####
1. Processes are given different size of quanta according to their priority. If they use up all quanta, they are moved down one class.

2. The UNIX system has a command `nice`, which allows a user to voluntarily reduce the priority of his process, in order to be kind to the other users. Nobody ever uses it.

#### Multiple Queues ####
Processes are divided into different classes where each class has its own scheduling needs. For example, a common division is a foreground (interactive) process and background (batch) processes. These two classes have different scheduling needs.

#### Shortest Process Next ####
n/a

#### Guaranteed Scheduling ####
This algorithm garantees fairness by monitorin gthe amount of CPU time spent by each user and allocating resources accordingly.

#### Lottery Scheduling ####
1. Give processes lottery tickets for various system resources, such as CPU time. Whenever a scheduling decision has to be made, a lotter ticket is chosen at random, and the process holdin gthat ticket gets the resource.

2. More important processes can be given extra tickets.

3. A process holdinga fraction *f* of the tickets will get about a fraction of *f* of the resource in question.

4. Cooperating processes may exchange tickets if they wish.

#### Fair-Share Scheduling ####
1. Each user is allocated some fraction of the CPU and the scheduler picks processes in such a way as to enforce it.

2. If two users have each been promised 50% of the CPU, they will each get that, no matter how many processes they have in existence.

### 2.4.4 Scheduling in Real-Time Systems ###
1. Real-time systems are divided into 
        - hard real time (there are absolute deadlines that must be met
        - soft real time (missing an occasional deadline is undesirable, but neverthelesse tolerable.

2. The events that a real-time system may have to respond to can be further categorized as 
        - periodic (meaning they occur at regular intervals)
        - aperiodic (they occur unpredictabley)
        
### 2.4.5 Policy Versus Mechanism ###
1. None of the schedulers discussed above accept any input from user processes about schedulign decisions. However, it is not uncommon that one process has many children runnign under its control and the parent process should know best which its child is the most important.

2. Seperating the ***scheduling mechanism*** from the ***scheduling policy*** is the key idea to solve the above question. The scheduling algorithm is parameterized in some way, but the parameters can be filled in by user processes.

### 2.4.6 Thread Scheduling ###
1. User-level threads vs. kernel-level threads

2. A major different between user-level threads and kernel-level threads is the performance. Doing a thread switch with user-level threads takes a handlful of machine instructions. With kernel-level threads it requires a full context switch, changing the memory map and invalidating the cache, which is several orders of magnitude slower.

3. On the other hand, with kernel-level threads, having a thread block on I/O does not suspend the entire process as it does with user-level threads.

4. Another important factor is that user-level threads can employ an applicaiton-specific thread scheduler.